<h2><b>[개인 과제]</b></h2>

````
pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/'
                     'wine/wine.data',header=None)
````
<h3>위와 같은 방식으로 다운받은 데이터에서 Class label이 1인 샘플과 Class label이 1이 아닌 샘플을 구분하게 하는 Binary Classifier를 만들어보세요!</h3>

<br>

**<h3><조건></h3>**

<h3>0. Train-test set 분리 </h3>

````
X_train, X_test, y_train, y_test = train_test_split(
X, y_1, test_size=0.33, random_state=64)
````
<h3>1. sklearn의 GridSearchCV를 활용하기</h3>

````
from sklearn.model_selection import GridSearchCV
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=0, scoring='f1')
````

<h3>2. grid-search에서 best-parameter, best score 확인하기</h3>
<h3>3. test set에서 f1 score와 confusion matrix 계산하기</h3>


# Binary Classifier 만들기

In [ ]:
# Importing common libraries
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [ ]:
# 파일 불러오기
dat_wine=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/'
                     'wine/wine.data',header=None)
dat_wine.columns = ['class label', 'alcohol', 'malic acid', 'ash', 
                    'alcalinity of ash', 'magnesium', 'total phenols', 
                    'flavanoids', 'nonflavanoid phenols', 
                    'proanthocyanins', 'color intensity', 'hue', 
                    'OD280', 'proline']  # Column names
print('class label:', np.unique(dat_wine['class label']))  # Class 출력
dat_wine.head()

class label: [1 2 3]


,class label,alcohol,malic acid,ash,alcalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD280,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
# 데이터 탐색 : null 값 없음
dat_wine.shape
dat_wine.info()
dat_wine.isna().any()

In [ ]:
# describe 둘러보기
dat_wine.describe()

,class label,alcohol,malic acid,ash,alcalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD280,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


# 0. split data & scale

In [ ]:
# split data
# X는 'class label' 제외한 attribute 들
# y는 알아내고 싶은 범주형 자료 'class label'
from sklearn.model_selection import  train_test_split
dat_wine_column = dat_wine.columns.to_list()
X = dat_wine[dat_wine_column[1:]]
y = dat_wine[dat_wine_column[0]]
y_1 = (y==1)
X_train, X_test, y_train, y_test = train_test_split(X, y_1, test_size=0.33, random_state=64)

In [ ]:
a = pd.Series(y_train)
print(a.value_counts()/len(a)) # 1/3정도 균형있게 split 된 듯

False    0.647059
True     0.352941
Name: class label, dtype: float64


In [ ]:
# 정규화 : 단위 척도가 다르므로 스케일링해주자
# 이때 X의 train set에는 .fit_transform 해주고 , test set에는 그냥 .transform 해줘 
# test 에도 fit 하면 안돼 
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train = ss.fit_transform(X_train) 
X_test = ss.transform(X_test) 

# 1. sklearn의 GridSearchCV를 활용하기
# 2. grid-search에서 best-parameter, best score 확인하기
# 3. test set에서 f1 score와 confusion matrix 계산하기

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [1,2,3,4,5,6,7,8,9,10]}]
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=0, scoring='f1')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [ ]:
#best-parameter 
print('GridSearchCV 최적 파라미터 : ', grid_search.best_params_)

GridSearchCV 최적 파라미터 :  {'n_neighbors': 4, 'weights': 'uniform'}


In [ ]:
# best score
print('GridSearchCV 최고 정확도 : ',grid_search.best_score_)

GridSearchCV 최고 정확도 :  0.9671826625386997


In [ ]:
# f1 score
from sklearn.metrics import f1_score
model = grid_search.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.9714285714285714

In [ ]:
#confusion_matrix 
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[41,  1],
       [ 0, 17]])